# Day 6

In [1]:
import numpy as np
import time

In [337]:
with open("input.txt", "r") as f:
    data = [line for line in f]

data = [int(n) for n in data[0].split(",")]
toy_data = [3,4,3,1,2]

## Puzzle 1

In [347]:
def sim_one_day(pop):
        
    pop_next_day = []
    
    for fish in pop:
        if fish == 0:
            pop_next_day.append(6)
            pop_next_day.append(8)
        else:
            pop_next_day.append(fish-1)

    return pop_next_day

def get_pop_size1(pop, days):
    
    for day in range(1, days+1):    

        pop_next_day = sim_one_day(pop)
        pop = pop_next_day
         
    print(f"After {days} days: {len(pop)}")

In [348]:
get_pop_size1(toy_data, 80)
get_pop_size1(data, 80)

After 80 days: 5934
After 80 days: 359344


## Puzzle 2

In [349]:
%%time
pop_growth = get_pop_size1(data, 80)

After 80 days: 359344
CPU times: user 486 ms, sys: 1.98 ms, total: 488 ms
Wall time: 487 ms


In [350]:
%%time

def sim_one_day2(n, new_gen):
    
    if n == 0:
        n = 6
        new_gen.append(8)
    else:
        n -= 1
        
    return n

vec_sim_one_day = np.vectorize(sim_one_day2, excluded=[1], otypes=[int])

def get_pop_size2(pop, days):

    pop = np.array(pop)
    new_gen = []

    for day in range(1, days+1):
        
        pop = vec_sim_one_day(pop, new_gen)

        if len(new_gen) > 0:
            pop = np.append(pop, new_gen)
            new_gen = []
    
    print(f"After {days} days: {len(pop)}")

get_pop_size2(data, 80)

After 80 days: 359344
CPU times: user 3.25 s, sys: 2.96 ms, total: 3.25 s
Wall time: 3.25 s


In [351]:
%%time

def get_pop_size3(pop, days):

    new_gen = np.empty(0)
    pop = np.array(pop)
    
    for day in range(1, days+1):
    
        pregnant_ix = pop == 0     
        new_gen = np.repeat(8, sum(pregnant_ix))
        pop[~pregnant_ix] = pop[~pregnant_ix] - 1
        pop[pregnant_ix] = 6
        pop = np.append(pop, new_gen)
        
    print(f"After {days} days: {len(pop)}")    

get_pop_size3(data, 80)

After 80 days: 359344
CPU times: user 6.79 s, sys: 978 µs, total: 6.79 s
Wall time: 6.79 s


#### Using lists seems to be faster, can't find a way to simulate all 256 days.. EPIC FAIL

## Elegant solution gently STOLEN (for my comprehension) from Joe

In [297]:
# Generate a dict with the pop of each fish state (num days from gen of new fish) after 128 days

fish_dict = {}

for j in range(0,9):
    fish = [j]
    
    for i in range(128):
        fish = sim_one_day(fish)
            
    fish_dict[j] = fish
    
# Obtain a dict with the pop size of each fish state after 128 days    
len_dict = {i:len(fish_dict[i]) for i in fish_dict}
len_dict

{0: 94508,
 1: 90763,
 2: 79321,
 3: 75638,
 4: 67616,
 5: 62475,
 6: 58016,
 7: 51564,
 8: 49380}

In [304]:
# For each fish state, iterate through each fish of each pop obtained after 128 days
# and get the pop size that each fish would have after another 128 days and sum them

len_dict_full = {}

# Iterate through the fish states
for state in range(9):
    
    length = 0
    
    # For each fish obtained after 128 days
    for fish in fish_dict[state]:
        
        # Obtain the pop size after another 128 days
        length += len_dict[fish]
    
    len_dict_full[state] = length

len_dict_full

{0: 6703087164,
 1: 6206821033,
 2: 5617089148,
 3: 5217223242,
 4: 4726100874,
 5: 4368232009,
 6: 3989468462,
 7: 3649885552,
 8: 3369186778}

In [306]:
# Sum the pop size that each fish present in the original pop would have after 256 days
sum([len_dict_full[i] for i in pop])

26984457539